# Datasets and Data Objects

## Datasets

First, import yt. Simple enough. 

In [ ]:
import yt

We can use the `load` function to load up one of our sample datasets. Let's load up the `IsolatedGalaxy` dataset:

In [ ]:
ds = yt.load('IsolatedGalaxy/galaxy0030/galaxy0030')

We can see that when we loaded the dataset, it parsed some parameters and informed us of what they were. We are interested in what kinds of "fields" are in the dataset. We can list them using `ds.field_list`:

In [ ]:
ds.field_list

We also mentioned "derived" fields earlier. We can see which derived fields are available using `ds.derived_field_list`:

In [ ]:
ds.derived_field_list

Notice that the fields that are in the dataset are part of the derived field list as well--reflecting yt's philosophy on fields.

Let's now get some basic stats on this dataset, which is a grid-based Enzo dataset. We can do that with the `ds.print_stats()` method:

In [ ]:
ds.print_stats()

Let's also look at some basic properties associated with this dataset: 

In [ ]:
print(ds.domain_left_edge)
print(ds.domain_right_edge)
print(ds.domain_width)
print(ds.current_time)

These are "code" units, which are the default units of the dataset. We'd probably like to see what the units are in terms of something more "physical". We can use the `in_units()` method for that:

In [ ]:
print(ds.domain_left_edge.in_units("Mpc"))
print(ds.domain_right_edge.in_units("Mpc"))
print(ds.domain_width.in_units("Mpc"))
print(ds.current_time.in_units("Myr"))

We'll spend more time with yt's unit system later. 

## Data Objects

Data objects in yt are 

### Spheres

Spheres, of course, are defined by a center and a radius. 

In [ ]:
dd = ds.all_data()

In [ ]:
dd[('gas','density')]

In [ ]:
sp = ds.sphere("max", (10, "kpc"))

In [ ]:
list(sp.quantities.keys())

In [ ]:
sp.quantities.total_mass()

In [ ]:
## NumPy-like operations

* yt has another way to create rectangular-shaped regions using NumPy-like syntax
* The resulting regions have useful NumPy-like operations

In [ ]:
dd = ds.r[:,:,:] # A region containing all the data

In [ ]:
reg = ds.r[0.4:0.6,0.1:0.3,0.35:0.75] # subset of the domain in code units

In [ ]:
reg.mean("density")

In [ ]:
c = ds.domain_center
w = 5.0*u.Mpc
le = c-0.5*w
re = c+0.5*w

In [ ]:
reg2 = ds.r[le[0]:re[0],le[1]:re[1],le[2]:re[2]] # subset of the domain using a left and right edge

In [ ]:
reg2.mean("density")

In [ ]:
def _emissivity(field, data):
    newfield = data[('gas','density')]**2*data[('gas','temperature')]**0.5
    return(newfield)

ds.add_field('emissivity', function = _emissivity, units="K**0.5*g**2/cm**6")
dd = ds.all_data()
print(dd['emissivity'])
                                               

In [ ]:
max_val, max_loc = ds.find_max(('gas','temperature'))
print(max_loc)
s = yt.SlicePlot(ds, "x", "density", center=max_loc, width=(100,"kpc"))
s.show()

In [ ]:
s.zoom(2.5)

In [ ]:
s.set_width((200,"kpc"))

In [ ]:
s.annotate_contour(('gas','temperature'))

In [ ]:
s.annotate_contour?

In [ ]:
s.annotate_contour(("gas","temperature"), take_log=False, clim=(1e4,1e6))

In [ ]:
s.set_width((400,"kpc"))

In [ ]:
s.annotate_clear()
s.annotate_velocity()
s.show()

In [ ]:
s.zoom(3)

In [ ]:
s.annotate_clear()
s.annotate_grids()
s.show()

In [ ]:
s.annotate_cell_edges()

In [ ]:
s.annotate_clear()
s.annotate_title('I am the best plot!')
s.annotate_text((20, 20), 'Galaxy!', coord_system='plot')

In [ ]:
s.annotate_clear()
s.hide_colorbar()
s.hide_axes()

In [ ]:
p = yt.ProjectionPlot(ds,'y','density')
p.show()

In [ ]:
pw = yt.ProjectionPlot(ds,'y', 'temperature', weight_field='density')
pw.set_cmap('temperature','jet')
pw.show()

In [ ]:
yt.show_colormaps()

In [ ]:
sp = ds.sphere("center", (15, "kpc"))
L = sp.quantities.angular_momentum_vector()
print(L)


In [ ]:
soff = yt.OffAxisSlicePlot(ds, L, "density", sp.center, (25,"kpc"))
soff.show()

In [ ]:
poff = yt.OffAxisProjectionPlot(ds, L, 'density', sp.center, (25,"kpc"))
poff.show()

In [ ]:
ph = yt.PhasePlot(sp, "density", "temperature", "cell_mass", weight_field=None)
ph.show()

In [ ]:
ph = yt.PhasePlot(sp, "density", "temperature", "H_fraction", weight_field=None)
ph.show()

In [ ]:
ph = yt.PhasePlot(sp, ('gas','density'), ('gas','temperature'), ('gas','H_fraction'))
ph.show()

In [ ]:
my_disk = ds.disk(ds.domain_center, [0.0, 0.0, 1.0], (10, "kpc"), (3, "kpc"))
p1d = yt.ProfilePlot(my_disk, "density", "temperature")
p1d.show()

In [ ]:
p1d_acc = yt.ProfilePlot(sp, "radius", "cell_mass", weight_field=None, accumulation=True)
p1d_acc.show()